In [1]:
import pandas as pd
import warnings
import json
from datetime import datetime
warnings.filterwarnings('ignore')
pd.options.display.float_format = '{:.2f}'.format

In [15]:
%%sql
SELECT 
    data,
    weekofyear(data) AS semana,
    numeroNotaFiscal, 
    destino, 
    procedencia, 
    cad_cidades.estado as estado,
    divtec_notasfiscais.produto,
    grupo,
    unidade, 
    unidade_peso, 
    quantidade, 
    volume,
    cad_usuarios.tipo,
    nome_usuario_sistema
FROM 
    divtec_notasfiscais
LEFT JOIN
    cad_usuarios
        ON cad_usuarios.matricula_nome = divtec_notasfiscais.destino
LEFT JOIN
    cad_cidades ON divtec_notasfiscais.procedencia = cad_cidades.cidade
LEFT JOIN divtec_produtos ON divtec_notasfiscais.produto = divtec_produtos.produto
WHERE data >= '2024-08-01' and data < '2024-09-01' and grupo = 'FLORES'

,data,semana,numeroNotaFiscal,destino,procedencia,estado,produto,grupo,unidade,unidade_peso,quantidade,volume,tipo,nome_usuario_sistema
0,2024-08-01,31,56073922,0100037-YOSHIHIRO TAKEUCHI,0143024-CANOAS (RS),RIO GRANDE DO SUL,12301-FLORES COMESTIVEIS,FLORES,UND,0.03,20.00,0.60,PRODUTOR,Deise
1,2024-08-06,32,333,0100037-YOSHIHIRO TAKEUCHI,0143024-CANOAS (RS),RIO GRANDE DO SUL,12301-FLORES COMESTIVEIS,FLORES,OUTROS,0.05,10.00,0.50,PRODUTOR,Nicolly
2,2024-08-08,32,56188699,0100037-YOSHIHIRO TAKEUCHI,0143241-PAROBE (RS),RIO GRANDE DO SUL,12301-FLORES COMESTIVEIS,FLORES,OUTROS,0.05,20.00,1.00,PRODUTOR,Nicolly
3,2024-08-09,32,90820001,0102230-MARIA INES DE JESUS,0143165-ALVORADA (RS),RIO GRANDE DO SUL,12301-FLORES COMESTIVEIS,FLORES,OUTROS,0.05,20.00,1.00,PRODUTOR,Jessica
4,2024-08-13,33,56253657,0100037-YOSHIHIRO TAKEUCHI,0143024-CANOAS (RS),RIO GRANDE DO SUL,12301-FLORES COMESTIVEIS,FLORES,UND,0.03,20.00,0.60,PRODUTOR,Deise
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
308,2024-08-23,34,4683,5500841-CONCEICAO DA ROSA MOLLER,0143213-PORTAO (RS),RIO GRANDE DO SUL,ZAMIOCULCA,FLORES,UND,2.00,4.00,8.00,ATACADISTA,Mateus
309,2024-08-26,35,3673999,5501050-BONET& FREITAS COM. VAR,0135480-SANTO ANTONIO DE POSSE (SP),SAO PAULO,ZAMIOCULCA,FLORES,UND,2.00,18.00,36.00,ATACADISTA,Simone
310,2024-08-26,35,4701,5500841-CONCEICAO DA ROSA MOLLER,0143213-PORTAO (RS),RIO GRANDE DO SUL,ZAMIOCULCA,FLORES,UND,2.00,4.00,8.00,ATACADISTA,Simone
311,2024-08-27,35,4702,5500841-CONCEICAO DA ROSA MOLLER,0143213-PORTAO (RS),RIO GRANDE DO SUL,ZAMIOCULCA,FLORES,UND,2.00,4.00,8.00,ATACADISTA,Luiz


In [16]:
df['data'] = df['data'].astype('datetime64[ns]')

## Semana

In [38]:
semana = df.copy(deep=True)

In [39]:
semana = semana.groupby(['semana','produto'])['volume'].sum().reset_index()
semana['semana'] = semana['semana'].astype('str')

In [41]:
semana_pivot = pd.pivot_table(data=semana, columns='semana',index='produto',values='volume')
semana_pivot['total'] = semana_pivot.sum(axis=1)
semana_pivot = semana_pivot.fillna(0)
semana_pivot

semana,31,32,33,34,35,total
produto,,,,,,
12301-FLORES COMESTIVEIS,0.60,2.50,3.88,21.00,5.70,33.68
50001-ANTURIO VASO,0.00,0.00,0.00,0.00,10.80,10.80
50102-ASTER,0.00,2.50,0.00,0.00,20.00,22.50
50202-ASTROMELIA,15.00,3940.00,1420.00,1440.00,610.00,7425.00
50401-BEGONIA,0.00,0.00,7.20,0.00,0.00,7.20
50801-CACTUS,0.00,2.10,0.00,57.30,6.30,65.70
50806-CACTUS MINI 15UN,16.00,92.00,40.00,52.00,50.00,250.00
51402-GERBERA 12UN,0.00,0.00,0.00,30.00,0.00,30.00
51501-GIRASSOL VASO,0.00,266.40,0.00,72.00,0.00,338.40


In [47]:
semana_pivot.to_excel('Dados Flores Semanal.xlsx')

## Cotação

In [48]:
%%sql
CALL CotacaoEmQuilogramasPorPeriodo('2024-08-01','2024-08-31')

,data,produto,unidade,mais_frequente,peso em KG,Preço por Kg
0,2024-08-01,00101-ABACATE,KG,5.55,1.00,5.55
1,2024-08-01,00201-ABACAXI CAIENA (HAWAI),UND,7.00,1.80,3.89
2,2024-08-01,00202-ABACAXI PEROLA,UND,6.00,1.30,4.62
3,2024-08-01,00350-AMEIXA IMPORTADA,KG,16.67,1.00,16.67
4,2024-08-01,00401-AMORA PRETA,KG,120.00,1.00,120.00
...,...,...,...,...,...,...
4827,2024-08-30,54901-ROSA COLORIDA 20UN,MOL,60.00,0.60,100.00
4828,2024-08-30,54950-ROSA IMPORTADA,MOL,0.00,0.60,0.00
4829,2024-08-30,FOLHAGENS VARIADAS,UND,20.00,0.00,NaN
4830,2024-08-30,MANGOSTAO,KG,50.00,1.00,50.00


In [51]:
cotacao = cotacao.groupby(['produto'])['Preço por Kg'].mean().reset_index()

## Valores Comercializados

In [53]:
valores = semana.merge(cotacao, on=['produto'], how='left')
valores['valor comercializado'] = valores['volume'] * valores['Preço por Kg']
valores

,semana,produto,volume,Preço por Kg,valor comercializado
0,31,12301-FLORES COMESTIVEIS,0.60,345.45,207.27
1,31,50202-ASTROMELIA,15.00,65.91,988.63
2,31,50806-CACTUS MINI 15UN,16.00,21.82,349.09
3,31,51701-CRISANTEMO,1.20,203.03,243.64
4,31,51702-CRISANTEMO VASO,3704.40,20.45,75770.13
...,...,...,...,...,...
95,35,53501-KALANDIVA VASO,18.00,26.44,475.91
96,35,53601-LIRIO DA PAZ,3.60,61.06,219.82
97,35,54601-ROSA VERMELHA 20UN,74.80,102.80,7689.17
98,35,54701-MOSQUITINHO,12.00,107.43,1289.11


In [54]:
valores_pivot = pd.pivot_table(data=valores, columns='semana',index='produto',values='valor comercializado')
valores_pivot['total'] = valores_pivot.sum(axis=1)
valores_pivot = valores_pivot.fillna(0)
valores_pivot

semana,31,32,33,34,35,total
produto,,,,,,
12301-FLORES COMESTIVEIS,207.27,863.63,1340.35,7254.49,1969.08,11634.82
50001-ANTURIO VASO,0.00,0.00,0.00,0.00,1210.89,1210.89
50102-ASTER,0.00,198.48,0.00,0.00,1587.86,1786.35
50202-ASTROMELIA,988.63,259680.03,93590.26,94908.44,40204.27,489371.62
50401-BEGONIA,0.00,0.00,439.64,0.00,0.00,439.64
50801-CACTUS,0.00,57.43,0.00,1567.05,172.29,1796.78
50806-CACTUS MINI 15UN,349.09,2007.27,872.73,1134.55,1090.91,5454.55
51402-GERBERA 12UN,0.00,0.00,0.00,2604.55,0.00,2604.55
51501-GIRASSOL VASO,0.00,7548.20,0.00,2040.05,0.00,9588.26


In [55]:
valores_pivot.to_excel('Flores Valores Comercializados.xlsx')

In [2]:
%%sql
SELECT
    data,
    weekofyear(data) AS semana,
    numeroNotaFiscal,
    destino,
    procedencia,
    cad_cidades.estado as estado,
    divtec_notasfiscais.produto,
    grupo,
    unidade,
    unidade_peso,
    quantidade,
    volume,
    cad_usuarios.tipo,
    nome_usuario_sistema
FROM
    divtec_notasfiscais
LEFT JOIN
    cad_usuarios
        ON cad_usuarios.matricula_nome = divtec_notasfiscais.destino
LEFT JOIN
    cad_cidades ON divtec_notasfiscais.procedencia = cad_cidades.cidade
LEFT JOIN divtec_produtos ON divtec_notasfiscais.produto = divtec_produtos.produto
WHERE EXTRACT(MONTH FROM data) = 5 and EXTRACT(YEAR FROM data) = 2023 and grupo = 'FLORES'

Exception: [42S22][1054] Unknown column 'grupo' in 'field list'